In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import random
import os
from PIL import Image

In [ ]:
train_dir = "data/train"
test_dir = "data/test"

In [ ]:
IMG_SIZE = (224, 224)
batchsize = 32
train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                                label_mode="categorical",
                                                                                batch_size = batchsize ,
                                                                                image_size=IMG_SIZE)
                                                                                
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE,
                                                                batch_size = batchsize ,
                                                                shuffle=False) 

In [4]:
checkpoint_path = "model_checkpoint"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True, 
                                                         monitor="val_accuracy", 
                                                         save_best_only=True) 

In [5]:
data_augmentation = Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.2),
  layers.RandomZoom(0.2),
  layers.RandomHeight(0.2),
  layers.RandomWidth(0.2),
  # preprocessing.Rescaling(1./255) # keep for ResNet50V2, remove for EfficientNetB0
], name ="data_augmentation")


In [6]:
base_model = tf.keras.applications.efficientnet.EfficientNetB4(include_top=False)
base_model.trainable = False
inputs = layers.Input(shape=(224, 224, 3), name="input_layer")
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D(name="global_average_pooling")(x)
outputs = layers.Dense(len(train_data.class_names), activation="softmax", name="output_layer")(x) 
model = tf.keras.Model(inputs, outputs)

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation (Sequenti  (None, 224, 224, 3)      0         
 al)                                                             
                                                                 
 efficientnetb4 (Functional)  (None, None, None, 1792)  17673823 
                                                                 
 global_average_pooling (Glo  (None, 1792)             0         
 balAveragePooling2D)                                            
                                                                 
 output_layer (Dense)        (None, 2)                 3586      
                                                                 
Total params: 17,677,409
Trainable params: 3,586
Non-trainabl

In [8]:
int(len(test_data))

100

In [9]:
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(), 
              metrics=["accuracy"])

history_all_classes_10_percent = model.fit(train_data,
                                           epochs=2, 
                                           steps_per_epoch = int(len(train_data)),
                                           validation_data=test_data,
                                           validation_steps=int(len(test_data)), 
                                           callbacks=[checkpoint_callback]) 

Epoch 1/2
400/400 [==============================] - 854s 2s/step - loss: 0.1350 - accuracy: 0.9576 - val_loss: 0.0595 - val_accuracy: 0.9953
Epoch 2/2
400/400 [==============================] - 943s 2s/step - loss: 0.0652 - accuracy: 0.9818 - val_loss: 0.0736 - val_accuracy: 0.9859


In [10]:
model.evaluate(test_data)

100/100 [==============================] - 177s 2s/step - loss: 0.0736 - accuracy: 0.9859


[0.07356280833482742, 0.9859374761581421]

In [15]:
model.save_weights("model_weights.h5")

In [16]:
# Save the model in SavedModel format
model.save("model_saved_model")

INFO:tensorflow:Assets written to: model_saved_model\assets


INFO:tensorflow:Assets written to: model_saved_model\assets


TypeError: Unable to serialize [2.0896919 2.1128857 2.1081853] to JSON. Unrecognized type <class 'tensorflow.python.framework.ops.EagerTensor'>.

In [8]:
load = model.load_weights("model_weights.h5")

In [ ]:
img_path = "data/testing/"
rand_img = random.choice(os.listdir(img_path))
img = img_path + rand_img
img = Image.open(img)
resize_img = img.resize((224 , 224))
image = np.array(resize_img)
plt.imshow(img)
plt.axis("off");